<a href="https://colab.research.google.com/github/victormenuzzo/amazon_satellite_imgs/blob/master/tag_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
from google.colab import drive
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [6]:
drive.mount('/content/drive')
image_tags = pd.read_csv('/content/drive/Shared drives/TCC/train.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
image_tags = image_tags.set_index('image_name')
image_tags.head()

,tags
image_name,
train_0,haze primary
train_1,agriculture clear primary water
train_2,clear primary
train_3,clear primary
train_4,agriculture clear habitation primary road


In [66]:
G = nx.MultiGraph()

for image_tag in image_tags['tags'].values:
  image_tag_list = image_tag.split(' ')
  
  k = 0 
  
  while k < len(image_tag_list)-1:
    actual_node = image_tag_list[k]

    for i in range (1, len(image_tag_list)-k):
      G.add_edge(actual_node, image_tag_list[k+i])

    k = k+1

In [95]:
df = pd.DataFrame(nx.to_numpy_matrix(G), index=G.nodes, columns = G.nodes())
df

,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
haze,0.0,2670.0,672.0,0.0,613.0,129.0,394.0,205.0,3.0,0.0,2.0,41.0,5.0,4.0,5.0,0.0
primary,2670.0,0.0,11972.0,27668.0,7001.0,3469.0,7728.0,4523.0,209.0,7175.0,94.0,683.0,324.0,332.0,340.0,101.0
agriculture,672.0,11972.0,0.0,9150.0,2712.0,2737.0,6034.0,3447.0,119.0,2493.0,24.0,225.0,38.0,32.0,65.0,23.0
clear,0.0,27668.0,9150.0,0.0,5502.0,3090.0,6295.0,3583.0,173.0,0.0,70.0,747.0,307.0,311.0,308.0,87.0
water,613.0,7001.0,2712.0,5502.0,0.0,915.0,2125.0,887.0,24.0,1295.0,26.0,206.0,299.0,16.0,49.0,3.0
habitation,129.0,3469.0,2737.0,3090.0,915.0,0.0,2786.0,925.0,41.0,441.0,36.0,163.0,29.0,4.0,13.0,3.0
road,394.0,7728.0,6034.0,6295.0,2125.0,2786.0,0.0,1337.0,36.0,1382.0,59.0,323.0,110.0,10.0,151.0,2.0
cultivation,205.0,4523.0,3447.0,3583.0,887.0,925.0,1337.0,70.0,130.0,759.0,4.0,89.0,18.0,35.0,58.0,8.0
slash_burn,3.0,209.0,119.0,173.0,24.0,41.0,36.0,130.0,0.0,33.0,0.0,10.0,0.0,2.0,2.0,2.0
partly_cloudy,0.0,7175.0,2493.0,0.0,1295.0,441.0,1382.0,759.0,33.0,0.0,28.0,74.0,27.0,17.0,27.0,14.0


In [134]:
df_total = pd.DataFrame(df.sum(axis=0), columns=['total'])
df_total = df_total.sort_values(by='total').reset_index()
df_total = df_total.loc[:5, 'index']
six_min_tags = df_total.values.tolist()

,total
haze,4743.0
primary,74289.0
agriculture,39743.0
clear,57291.0
water,21673.0
habitation,14781.0
road,28772.0
cultivation,16078.0
slash_burn,784.0
partly_cloudy,13765.0


In [128]:
df_min_tags = pd.DataFrame(columns = ['tag', 'image_tags'])

for tag in six_min_tags: 
  for image_tag in image_tags['tags'].values:
     if tag in image_tag:
        line = {'tag':[tag], 'image_tags':[image_tag]}
        df_min_tags = df_min_tags.append(pd.DataFrame.from_dict(line), ignore_index = True)


In [135]:
df_min_tags.set_index('tag')

,image_tags
tag,
blow_down,blow_down clear primary
blow_down,bare_ground blow_down clear primary slash_burn
blow_down,blow_down clear primary
blow_down,blow_down clear primary
blow_down,blow_down clear primary
...,...
artisinal_mine,artisinal_mine bare_ground clear primary water
artisinal_mine,artisinal_mine clear primary road water
artisinal_mine,artisinal_mine clear habitation primary road w...
